# Spam classification with BERT

In [2]:
!pip install tensorflow_text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 4.8 MB/s 
     |████████████████████████████████| 588.3 MB 15 kB/s 
     |████████████████████████████████| 6.0 MB 59.3 MB/s 
     |████████████████████████████████| 439 kB 63.9 MB/s 
     |████████████████████████████████| 1.7 MB 53.0 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 1.1

## Importing important packages

In [1]:
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd

## Dataset

In [5]:
%cd /content/drive/MyDrive/Colab Notebooks/

/content/drive/MyDrive/Colab Notebooks


In [6]:
df = pd.read_csv("spam.csv")

In [7]:
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [8]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [9]:
747/4825

0.15481865284974095

## Balancing dataset

In [10]:
df_spam = df[df['Category']=='spam']

In [11]:
df_ham = df[df['Category']=='ham']

In [12]:
df_ham_downsampled = df_ham.sample(df_spam.shape[0])

In [13]:
df_balanced = pd.concat([df_ham_downsampled, df_spam])

In [17]:
df_balanced['Category'].value_counts()

ham     747
spam    747
Name: Category, dtype: int64

## Adding labels

In [18]:
df_balanced['spam']=df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)

In [20]:
df_balanced.sample(5)

,Category,Message,spam
4658,spam,PRIVATE! Your 2003 Account Statement for shows...,1
5364,spam,Call 09095350301 and send our girls into eroti...,1
4226,ham,The world suffers a lot... Not because of the ...,0
5378,spam,Free entry to the gr8prizes wkly comp 4 a chan...,1
1549,ham,"Me, i dont know again oh",0


## Splitting labeled dataset

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(df_balanced['Message'],df_balanced['spam'], stratify=df_balanced['spam'])

## BERT

In [23]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

In [25]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [26]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 text (InputLayer)              [(None,)]            0           []                               
                                                                                                  
 keras_layer (KerasLayer)       {'input_mask': (Non  0           ['text[0][0]']                   
                                e, 128),                                                          
                                 'input_word_ids':                                                
                                (None, 128),                                                      
                                 'input_type_ids':                                                
                                (None, 128)}                                                  

In [28]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [29]:
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
35/35 [==============================] - 516s 14s/step - loss: 0.6492 - accuracy: 0.6357 - precision: 0.6333 - recall: 0.6446
Epoch 2/10
35/35 [==============================] - 499s 14s/step - loss: 0.5087 - accuracy: 0.8223 - precision: 0.8065 - recall: 0.8482
Epoch 3/10
35/35 [==============================] - 495s 14s/step - loss: 0.4364 - accuracy: 0.8634 - precision: 0.8564 - recall: 0.8732
Epoch 4/10
35/35 [==============================] - 495s 14s/step - loss: 0.3882 - accuracy: 0.8795 - precision: 0.8632 - recall: 0.9018
Epoch 5/10
35/35 [==============================] - 495s 14s/step - loss: 0.3567 - accuracy: 0.8893 - precision: 0.8707 - recall: 0.9143
Epoch 6/10
35/35 [==============================] - 498s 14s/step - loss: 0.3381 - accuracy: 0.8893 - precision: 0.8746 - recall: 0.9089
Epoch 7/10
35/35 [==============================] - 494s 14s/step - loss: 0.3171 - accuracy: 0.8982 - precision: 0.8805 - recall: 0.9214
Epoch 8/10
35/35 [=======================

In [30]:
y_predicted = model.predict(X_test)
y_predicted = y_predicted.flatten()

12/12 [==============================] - 168s 14s/step


In [31]:
import numpy as np

y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

array([1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0,
       0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 1,
       1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0,
       0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1,

In [32]:
sample_dataset = [
    'You can win alot of money, register in the link below',
    'You have an iphone 10, spin the image below to claim your prize and it willl be delivered in your door step',
    'You have an offer, the company will give you 50% off in every item purchased.',
    'Hey Bravin, dont be late for the meeting tomorrow, it will start at exactly 10:30am',
    "See you monday, we have alot to talk about the future of this company ."
]
model.predict(sample_dataset)

SyntaxError: ignored